In [106]:
import pandas as pd
import datetime

In [107]:
data = pd.read_csv('5-8 Step Funnel_Payment (5).csv')
data.head()

,Event,Date,Event Value,Email,Device Id,Oct 1 2018 - Jan 22 2019
0,screen-view - Unique,Dec 01 '18,confirm-tickets-page,claretburns84@live.co.uk,167696b24c70-0351bddb53c4b8-1a475464-410a0-167...,1
1,screen-view - Unique,Dec 01 '18,confirm-tickets-page,dionneblin@hotmail.com,undefined,1
2,screen-view - Unique,Dec 01 '18,confirm-tickets-page,hannahfeldman@me.com,undefined,1
3,screen-view - Unique,Dec 01 '18,confirm-tickets-page,harriscaroline41@gmail.com,1676ad450ba21-014dca1ff2a758-59175b1d-38400-16...,1
4,screen-view - Unique,Dec 01 '18,confirm-tickets-page,lindsay.read@hotmail.co.uk,1676938c16d346-043a2d77866bbf8-704b1e6f-3d10d-...,1


In [108]:
data['Event'] = data['Event'].apply(lambda s: s.split(' - Unique')[0])
data['Date'] = data['Date'].apply(lambda x: datetime.datetime.strptime(x, '%b %d \'%y')
                                              .strftime('%Y%m%d'))
#data = data[data['mp_processing_time_ms']!='undefined']
#data['date'] = data['mp_processing_time_ms'].apply(lambda ms: datetime.datetime.fromtimestamp(int(ms)/1000.0).strftime('%Y%m%d'))
data.head()

,Event,Date,Event Value,Email,Device Id,Oct 1 2018 - Jan 22 2019
0,screen-view,20181201,confirm-tickets-page,claretburns84@live.co.uk,167696b24c70-0351bddb53c4b8-1a475464-410a0-167...,1
1,screen-view,20181201,confirm-tickets-page,dionneblin@hotmail.com,undefined,1
2,screen-view,20181201,confirm-tickets-page,hannahfeldman@me.com,undefined,1
3,screen-view,20181201,confirm-tickets-page,harriscaroline41@gmail.com,1676ad450ba21-014dca1ff2a758-59175b1d-38400-16...,1
4,screen-view,20181201,confirm-tickets-page,lindsay.read@hotmail.co.uk,1676938c16d346-043a2d77866bbf8-704b1e6f-3d10d-...,1


In [109]:
data['Event Value'] = data.apply(lambda row: row['Event Value'] if row['Event']=='screen-view' else '', axis=1)

In [110]:
event_values_to_use = ['your-tickets-page','select-your-seats-page', 'confirm-tickets-page', 'payment-page']
data['to_drop'] = data.apply(lambda row: 1 if (row['Event']=='screen-view' and row['Event Value'] not 
                                               in event_values_to_use) else 0, axis=1)

In [111]:
data = data[data['to_drop']!=1]

In [112]:
data['Event'] = data.apply(lambda row: row['Event Value'] if row['Event']=='screen-view' else row['Event'], axis=1)

In [113]:
data['Email+device_id'] = data.apply(lambda row: ''.join([str(row['Email']),str(row['Device Id'])]), axis=1)

In [114]:
result = data.groupby(['Date', 'Event'])['Email+device_id'].apply(list).reset_index()
result['Email+device_id'] = result['Email+device_id'].apply(lambda x: len(list(set(x))))
result.head()

,Date,Event,Email+device_id
0,20181001,confirm-tickets-page,7
1,20181001,payment-page,7
2,20181001,select-your-seats-page,7
3,20181001,your-tickets-page,14
4,20181002,confirm-tickets-page,24


In [115]:
index = {'confirm-tickets-page':4,
 'pay-button-pressed':6,
 'pay-success':7,
 'payment-page':5,
 'select-your-seats-page':3,#?
 'selected-ticket-info':2,
 'your-tickets-page':1}
result['index'] = result['Event'].apply(lambda x: index[x])

In [116]:
#result.sort_values(by=['Date', 'index'], ascending=True)

In [117]:
result = result.sort_values(by=['Date', 'index'], ascending=True)
result.to_csv('5-8 steps formated data (3).csv')

In [51]:
test = result[result['Date'] > '20180917']
test.sort_values(by=['Date', 'index'], ascending=True)[:30]

,Date,Event,Email+device_id,index
1811,20180916,your-tickets-page,6,1
1810,20180916,selected-ticket-info,4,2
1809,20180916,select-your-seats-page,1,3
1807,20180916,pay-button-pressed,1,6
1808,20180916,pay-success,1,7
1816,20180917,your-tickets-page,6,1
1815,20180917,selected-ticket-info,6,2
1814,20180917,select-your-seats-page,1,3
1812,20180917,confirm-tickets-page,1,4
1813,20180917,payment-page,1,5


In [55]:
test = data[data['Date'] > '20180917']
l = ['your-tickets-page', 'selected-ticket-info']
test = test[test['Event'].isin(l)]
test.sort_values(by=['Date', 'Email+device_id'], ascending=True)[:50]

,Event,Email,Device Id,Date,Event Value,Sep 28 2017 - Jan 22 2019,to_drop,Email+device_id
454328,selected-ticket-info,ajita_d@yahoo.com,NaN,20180918,,1,0,ajita_d@yahoo.comnan
21757,your-tickets-page,britta@taraswonderworld.co.uk,NaN,20180918,your-tickets-page,1,0,britta@taraswonderworld.co.uknan
455156,selected-ticket-info,britta@taraswonderworld.co.uk,NaN,20180918,,1,0,britta@taraswonderworld.co.uknan
43460,your-tickets-page,charlotteamelia14@gmail.com,NaN,20180918,your-tickets-page,1,0,charlotteamelia14@gmail.comnan
455400,selected-ticket-info,charlotteamelia14@gmail.com,NaN,20180918,,1,0,charlotteamelia14@gmail.comnan
455988,selected-ticket-info,ehederer@gmail.com,NaN,20180918,,1,0,ehederer@gmail.comnan
456122,selected-ticket-info,emmaphelps@hotmail.co.uk,NaN,20180918,,1,0,emmaphelps@hotmail.co.uknan
456286,selected-ticket-info,feriel59@yahoo.co.uk,NaN,20180918,,1,0,feriel59@yahoo.co.uknan
456480,selected-ticket-info,gintare100@yahoo.com,NaN,20180918,,1,0,gintare100@yahoo.comnan
456662,selected-ticket-info,hannahfeldman@me.com,NaN,20180918,,1,0,hannahfeldman@me.comnan


In [60]:
data[data['Date']=='Sep 4 \'18']
#t[t['Date']=='Sep 4 \'18']

,Event,Email,Device Id,Date,Event Value,Sep 28 2017 - Jan 22 2019


In [ ]:
steps = {'step1': ['First App Open'],
         'step2':['successful-signup', 'successful-signin', 'successful-signinup'],
         #'step3':['presented-onboarding', 'selected-interest'],
         'step4':['viewed-collection','viewed-event', 'collection-venue-selected', 'search-performed', 'filter-pressed', 'viewed-category', 'viewed-venue'],
         'step5':['click-to-buy'],
         'step6':['selected-ticket-info'],
         'step7':['pay-button-pressed'],
         'step8':['pay-success']}

In [26]:
result[result['Event+']]

(20709, 7)

In [378]:
# drop undefined
#data['to_delete'] = data.apply(lambda row: 1 if (row['Email']=='undefined' and row['Device Id']=='undefined') else 0,
#                               axis=1)
#data = data[data['to_delete']!=1]
#data[data['to_delete']==1].shape[0]
#data = data[data['Email']!='undefined' && data['Device Id']!='undefined']
#data = data[data['Device Id']!='undefined']

In [379]:
steps = {'step1': ['First App Open'],
         'step2':['successful-signup', 'successful-signin', 'successful-signinup'],
         #'step3':['presented-onboarding', 'selected-interest'],
         'step4':['viewed-collection','viewed-event', 'collection-venue-selected', 'search-performed', 'filter-pressed', 'viewed-category', 'viewed-venue'],
         'step5':['click-to-buy'],
         'step6':['selected-ticket-info'],
         'step7':['pay-button-pressed'],
         'step8':['pay-success']}

In [380]:
for step in steps.keys():
    data['Event'] = data['Event'].apply(lambda event: step if event in steps[step] else event)

In [381]:
data['Email+user_id'] = data.apply(lambda row: ''.join([str(row['Email']),str(row['Device Id'])]), axis=1)

In [382]:
#step3 = ['presented-onboarding', 'selected-interest']
step3 = ['presented-onboarding','selected-interest']
temp = data[data['Event'].isin(step3)]
temp = temp.groupby(['Email+user_id', 'Date'])['Event'].apply(list).reset_index()
temp['Event'] = temp['Event'].apply(lambda l: list(set(l)))
temp['step3'] = temp['Event'].apply(lambda l: 1 if len(l)==2 else 0)
temp = temp[temp['step3']==1]

step3_ids = temp['Email+user_id'].values

In [383]:
data['Event'] = data.apply(lambda row: 'step3' if (row['Event'] in step3 and row['Email+user_id'] in step3_ids)
                                                  else row['Event'], axis=1)

In [384]:
steps['step3'] = []
data = data[data['Event'].isin(steps.keys())]
#data = data.drop('mp_processing_time_ms', 1)

In [385]:
result = data.drop_duplicates(subset=data.columns, keep='first')

In [386]:
installs = result[result['Event']=='step1']
installs = installs.sort_values(by=['Date'], ascending=True)
#instals.goupby(['Email+user_id'])['date'].
installs = installs.groupby(['Email+user_id'])['Date'].first()
installs = installs.to_dict()
installs.pop('nannan')

'20171201'

In [387]:
result['date_of_install'] = result.apply(lambda row: installs[row['Email+user_id']] 
                                         if row['Email+user_id'] in installs.keys()
                                         else 'unknown', axis=1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [388]:
result = result[result['date_of_install']!= 'unknown']
result.head()

,Event,Email,Device Id,Date,Dec 1 2017 - Dec 31 2018,Email+user_id,date_of_install
194,step5,alanasommers@hotmail.com,NaN,20180821,1,alanasommers@hotmail.comnan,20180115
195,step5,alanavidler@hotmail.com,NaN,20181206,1,alanavidler@hotmail.comnan,20180718
196,step5,alanavidler@hotmail.com,NaN,20181121,1,alanavidler@hotmail.comnan,20180718
197,step5,alanavidler@hotmail.com,NaN,20181129,1,alanavidler@hotmail.comnan,20180718
198,step5,alasadi_h@hotmail.co.uk,NaN,20180724,1,alasadi_h@hotmail.co.uknan,20180217


In [389]:
#result = result.groupby(['Date', 'Event'])['Email+user_id'].nunique().reset_index()
result = result.groupby(['date_of_install', 'Event'])['Email+user_id'].apply(list).reset_index()
result['Email+user_id'] = result['Email+user_id'].apply(lambda x: len(list(set(x))))
result.head()

,date_of_install,Event,Email+user_id
0,20171201,step1,17
1,20171201,step2,10
2,20171201,step3,14
3,20171201,step4,15
4,20171201,step5,6


In [390]:
result = result.sort_values(by=['date_of_install', 'Event'], ascending=True)


In [394]:
test = result[result['date_of_install'] > '20180515']
test.sort_values(by=['date_of_install', 'Event'], ascending=True)[:30]

,date_of_install,Event,Email+user_id
1153,20180516,step1,7
1154,20180516,step2,6
1155,20180516,step3,5
1156,20180516,step4,6
1157,20180516,step5,1
1158,20180517,step1,9
1159,20180517,step2,9
1160,20180517,step3,8
1161,20180517,step4,8
1162,20180517,step5,1


In [343]:
virgin = pd.read_csv('1 Step Funnel_Final.csv')
virgin['Event'] = virgin['Event'].apply(lambda s: s.split(' - Unique')[0])
virgin['Date'] = virgin['Date'].apply(lambda x: datetime.datetime.strptime(x, '%b %d \'%y')
                                              .strftime('%Y%m%d'))
virgin = virgin.sort_values(by=['Date'], ascending=True)
virgin[virgin['Event']=='presented-onboarding']

,Event,Email,Device Id,Date,Dec 1 2017 - Dec 31 2018
29060,presented-onboarding,acotsen@gmail.com,NaN,20180122,1
34142,presented-onboarding,kwds1378@gmail.com,NaN,20180122,1
34100,presented-onboarding,ksanc18@aol.com,NaN,20180122,1
36161,presented-onboarding,missbethell@yahoo.co.uk,NaN,20180122,1
37501,presented-onboarding,NaN,NaN,20180122,2
29419,presented-onboarding,alkisghinis@gmail.com,NaN,20180122,1
41088,presented-onboarding,ushamistry09@gmail.com,NaN,20180122,1
34678,presented-onboarding,lindsay_stewart123@hotmail.com,NaN,20180122,1
41412,presented-onboarding,vivekvpandey.vp@gmail.com,NaN,20180122,1
36935,presented-onboarding,nicolachennells@hotmail.com,NaN,20180122,1


In [ ]:
data = pd.read_csv('1 Step Funnel_Final.csv')
data.head()

In [346]:
result.to_csv('formated_data.csv',index=False)

In [ ]:
result_df

In [53]:
cols_to_merge = ['Date', 'Event']
final_data = pd.DataFrame()
for event in all_events:
    temp = data[data['Event']== event]
    temp = temp[['Country', 'City', 'Date', 'Value']]
    temp.columns = ['Country', 'City', 'Date', event]
    
    if final_data.empty:
        final_data = pd.DataFrame(temp)
    else:
        final_data = pd.merge(final_data, temp,  how='outer', on=cols_to_merge)

In [248]:
unique = list(set(data['Event'].values))
unique

['selected-ticket-info',
 'successful-signup',
 'viewed-event',
 'viewed-venue',
 'pay-button-pressed',
 'search-performed',
 'viewed-collection',
 'presented-onboarding',
 'First App Open',
 'pay-success',
 'click-to-buy',
 'filter-used',
 'successful-signin',
 'viewed-category',
 'successful-signinup',
 'selected-interest']

In [17]:
temp = data[data['Email']=='undefined']
temp = temp[temp['Device Id']=='undefined']
#temp = temp[temp['Event']!='viewed-collection']
#temp = temp[temp['Event']!='successful-signup']
temp.shape[0]/data.shape[0]

0.056380139767767734

In [25]:
d = list(set(data['Event'].values))
d = [str(x) for x in d]
d.sort()
d

['confirm-tickets-page',
 'pay-button-pressed',
 'pay-success',
 'payment-page',
 'select-your-seats-page',
 'selected-ticket-info',
 'your-tickets-page']

In [18]:
l = ['your-tickets-page','select-your-seats-page', 'confirm-tickets-page', 'payment-page']
t = list(filter(lambda x: 'your-tickets-page' in x, d))
t

['your-tickets-page']